In [1]:
from pyspark.ml.feature import BucketedRandomProjectionLSH
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("ApproximateNN").getOrCreate()

24/12/22 17:47:54 WARN Utils: Your hostname, Anls-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.21 instead (on interface en0)
24/12/22 17:47:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/22 17:47:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = [
    (0, Vectors.dense([0.1, 0.2, 0.3])),
    (1, Vectors.dense([0.4, 0.5, 0.6])),
    (2, Vectors.dense([0.7, 0.8, 0.9])),
    # Add more data as needed
]
df = spark.createDataFrame(data, ["id", "embedding"])

In [4]:
lsh = BucketedRandomProjectionLSH(
    inputCol="embedding",
    outputCol="hashes",
    bucketLength=2.0,
    numHashTables=3
)
model = lsh.fit(df)

24/12/22 17:47:57 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [5]:
query_vector = Vectors.dense([0.15, 0.25, 0.35])

In [6]:
result = model.approxNearestNeighbors(df, query_vector, 5)
result.select("id", "distCol").show()

+---+-------------------+
| id|            distCol|
+---+-------------------+
|  0|0.08660254037844384|
|  1| 0.4330127018922193|
|  2| 0.9526279441628825|
+---+-------------------+

